# Predicting Used Car Prices with Linear Regression
## Introduction


---
### Dataset Description
The dataset contains information about used cars, including features such as:

**Year:** The manufacturing year of the car.
**Kilometers_Driven:** Total distance driven (in kilometers).
**Fuel_Type:** Type of fuel (e.g., Petrol, Diesel, CNG, LPG).
**Transmission:** Manual or Automatic.
**Owner_Type:** Ownership history (e.g., First, Second).
**Mileage:** Fuel efficiency (e.g., kmpl or km/kg).
**Engine:** Engine displacement (in CC).
**Power:** Engine power (in bhp).
**Seats:** Number of seats.
**Price:** Target variable (price in lakhs).
**New_Price:** Price of the brand new car of the same characteristics.